In [1]:
#First of all, let's import necessary libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
#Let's read two datasets
#First of them is info dataset explaining variables
#Pay attention to what we indexed as index_column
#The other is our main dataset
df_info = pd.read_csv('lending_club_info.csv',index_col='LoanStatNew')
df = pd.read_csv('lending_club_loan_two.csv')

In [3]:
#Let's create function that displays description of a variable
#first of all let's how we can access description
print(df_info.loc['term']['Description'])

The number of payments on the loan. Values are in months and can be either 36 or 60.


In [4]:
#As we already understand how to extract desciption let's create 
#function which will give us description once we input variable_name

def description(variable_name):
    print(df_info.loc[variable_name]['Description'])

description('term')

The number of payments on the loan. Values are in months and can be either 36 or 60.


In [5]:
#some exploration
df.describe(include='all').T.sort_values('unique')
#From first two rows we can infer that almost from 40.000 observations 32.000
#are fully paid. The same ratio is applicable to term: most of the loans taken
#for 36 months

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
term,396030,2,36 months,302005,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_status,396030,2,Fully Paid,318357,NaN,NaN,NaN,NaN,NaN,NaN,NaN
initial_list_status,396030,2,f,238066,NaN,NaN,NaN,NaN,NaN,NaN,NaN
verification_status,396030,3,Verified,139563,NaN,NaN,NaN,NaN,NaN,NaN,NaN
application_type,396030,3,INDIVIDUAL,395319,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_ownership,396030,6,MORTGAGE,198348,NaN,NaN,NaN,NaN,NaN,NaN,NaN
grade,396030,7,B,116018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emp_length,377729,11,10+ years,126041,NaN,NaN,NaN,NaN,NaN,NaN,NaN
purpose,396030,14,debt_consolidation,234507,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub_grade,396030,35,B3,26655,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Should we drop emp_title variable? Let's examine
df['emp_title'].value_counts().head(20).sum()/len(df)
# as we can see, sum of 20 (more than that creation of dummy variables for
# for one variable is meaningless) is just 7%, for the rest we had to put OTHER
# label, so other label will be very strong and not intepretable
# So, we have to drop emp_title variable

0.07714567078251648

In [7]:
#what about title variable? only 10 highly frequent variables sum is 66%
#so, we have to preserve this variable and convert to dummy variables
#make sure, this variable does not have empty variable
df['title'].value_counts().head(10).sum()/len(df)

0.6609145771784966

In [8]:
df['title'].value_counts().head(10)

Debt consolidation         152472
Credit card refinancing     51487
Home improvement            15264
Other                       12930
Debt Consolidation          11608
Major purchase               4769
Consolidation                3852
debt consolidation           3547
Business                     2949
Debt Consolidation Loan      2864
Name: title, dtype: int64

In [9]:
#let's extractvmost frequent title values to iterate over
title = np.array(df['title'].value_counts().head(10).index)
title

array(['Debt consolidation', 'Credit card refinancing',
       'Home improvement', 'Other', 'Debt Consolidation',
       'Major purchase', 'Consolidation', 'debt consolidation',
       'Business', 'Debt Consolidation Loan'], dtype=object)

In [10]:
#let's turn into category dtype for optimization
df['title'] = df['title'].astype('category')

lst = []

for i in df['title']:
    if i in title :
        i=i
    else:
        i='different' #other than top 10 will be called different
    lst.append(i)

set(lst)

{'Business',
 'Consolidation',
 'Credit card refinancing',
 'Debt Consolidation',
 'Debt Consolidation Loan',
 'Debt consolidation',
 'Home improvement',
 'Major purchase',
 'Other',
 'debt consolidation',
 'different'}

In [11]:
#in a list above we see some similar titles, let's combine them

lst2 = ['Debt Consolidation','Debt Consolidation Loan',
       'Debt consolidation', 'debt consolidation','Consolidation']

lst3 = []

for i in lst:
    if i in lst2:
        i = 'Debt Consolidation'
    else:
        i = i
    lst3.append(i)

df['title'] = lst3 #variable joined to the main dataframe

In [12]:
#Three cells, includicng this, are very assissting, 
#just change var names and work (in the next two cells)
#they will hint you whether to drop or not
df.select_dtypes(['object']).columns

Index(['term', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'purpose', 'title', 'earliest_cr_line', 'initial_list_status',
       'application_type', 'address'],
      dtype='object')

In [13]:
df['application_type'].value_counts()

INDIVIDUAL    395319
JOINT            425
DIRECT_PAY       286
Name: application_type, dtype: int64

In [14]:
description('application_type')

Indicates whether the loan is an individual application or a joint application with two co-borrowers


In [15]:
#Variables to be dropped are 
df.drop(['address',         #too many adresses
        'emp_title',        #employee titles are very diverse, discussed above
        'title',            #processed above
        'earliest_cr_line', #for now no time-series analysis
        'issue_d',          #for now no time-series analysis,
        'application_type', #frequency one of values is higher than 95% of cases
        'sub_grade'],       #grade has same meaning and grouped as well
        inplace=True, axis=1)

# If you want to see value count of each variable

In [16]:
# Select ALl (ctrl+A) and press (ctrl+/) to decomment all
# #Now we have to see which are categorical datatypes to be converted to dummies
# #first of all let's see value counts of each variable

# colnames = df.columns #column names

# value_count = [] #empty list where value_count of each variable will be appende

# for i in df.columns:
#     value_count.append(df[i].value_counts().count())
    

# df_value_count = pd.DataFrame()
# df_value_count['Column_Name'] = colnames
# df_value_count['Value_Count'] = value_count
# df_value_count.sort_values('Value_Count', inplace=True)
# df_value_count

In [17]:
#save first parf of our project
df.to_csv("1_Lending_Club_final_variables.csv")